In [1]:
import tkinter as tk
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageTk
from image_conversor import convert_image
import cv2
import sys

# Substitua esta matriz de exemplo pela sua matriz de strings ASCII
ascii_matrix = np.array([
    [" @ ", " # ", " $ ", " % ", " & "],
    [" * ", " A ", " B ", " C ", " * "],
    [" * ", " D ", " E ", " F ", " * "],
    [" * ", " G ", " H ", " I ", " * "],
    [" * ", " * ", " * ", " * ", " * "]])

cap = cv2.VideoCapture('./data/Peixes.mp4')
_, frame = cap.read()
frame = convert_image(frame, 9)

# Criar a janela do tkinter
root = tk.Tk()
root.title("Arte ASCII")

# Função para criar e exibir a matriz como rótulos na janela
def display_ascii_matrix():
    return "\n".join("".join(row) for row in frame)

# Criar um rótulo para exibir a matriz na janela
label = tk.Label(root, text=display_ascii_matrix(), font=("Courier New", 12))
label.pack()

root.mainloop()

In [3]:
import tkinter as tk
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageTk
from image_conversor import convert_image
import cv2
import sys

# Substitua esta matriz de exemplo pela sua matriz de strings ASCII
ascii_matrix = np.array([
    ["@", "#", "$", "%", "&"],
    ["*", "A", "B", "C", "*"],
    ["*", "D", "E", "F", "*"],
    ["*", "G", "H", "I", "*"],
    ["*", "*", "*", "*", "*"]])

# Criar a janela do tkinter
root = tk.Tk()
root.title("Arte ASCII")

cap = cv2.VideoCapture('./data/Peixes.mp4')
_, frame = cap.read()
frame = convert_image(frame, 9)

# Função para formatar a matriz como uma string de matriz
def format_matrix(matrix):
    return '\n'.join([' '.join(row) for row in matrix])

# Criar um rótulo para exibir a matriz na janela
label = tk.Label(root, text=format_matrix(frame), font=("Courier New", 12))
label.pack()

root.mainloop()


In [18]:
import tkinter as tk
import numpy as np

# Substitua esta matriz de exemplo pela sua matriz de strings ASCII
ascii_matrix = np.array([
    ["@", "#", "$", "%", "&"],
    ["*", "A", "B", "C", "*"],
    ["*", "D", "E", "F", "*"],
    ["*", "G", "H", "I", "*"],
    ["*", "*", "*", "*", "*"]])

# Criar a janela do tkinter
root = tk.Tk()
root.title("Arte ASCII")
root.configure(bg="black")  # Definir o background para preto


cap = cv2.VideoCapture('./data/Peixes.mp4')
_, frame = cap.read()
frame = convert_image(frame, 10)


# Função para formatar a matriz como uma string de matriz
def format_matrix(matrix):
    return '\n'.join([' '.join(row) for row in matrix])

# Criar um rótulo para exibir a matriz na janela
label = tk.Label(root, text=format_matrix(frame), font=("Courier New", 10), bg="black", fg="#777777")  # Definir o background para preto e caracteres para branco
label.pack()

root.mainloop()
